In [ ]:
#make a virtual environment
# then, change your python kernel to your virtual environment
!py -3.10 -m venv env

In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124
!py -m pip install --upgrade pip
!pip install ultralytics
!pip install YOLO


# Note. if you have NVIDIA, you can install the following packages to use GPU
# You can go to pytorch.org to get the correct version of the package
# you also need to install CUDA and cuDNN
# after installing, you can look your cuda version by running `nvcc --version` from your command line
# Cuda is backward compatible, so you can install the latest version of CUDA and use the oldest version of pytorch
# pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

Looking in indexes: https://download.pytorch.org/whl/cu124


In [7]:
# Remove images without corresponding labels

import os

# Define the folder paths
images_folder = r'crocDataaset\train\images' 
labels_folder = r'crocDataaset\train\labels'

# Get lists of image and label file names
image_files = set(os.listdir(images_folder))
label_files = set(os.listdir(labels_folder))

# Extract base names without extensions
image_bases = {os.path.splitext(file)[0] for file in image_files}
label_bases = {os.path.splitext(file)[0] for file in label_files}

# Find images without corresponding labels
images_to_remove = image_bases - label_bases

# Remove unmatched images
for image in images_to_remove:
    image_path = os.path.join(images_folder, f"{image}.jpg")  # Adjust extension as needed
    if os.path.exists(image_path):
        os.remove(image_path)
        print(f"Removed: {image_path}")


In [3]:
%pip install scikit-learn

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ----------------------------------- ---- 9.7/11.0 MB 50.2 MB/s eta 0:00:01
   ---------------------------------------- 11.0/11.0 MB 43.0 MB/s eta 0:00:00
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
#split the dataset into train, val, and test sets = 70%, 20%, 10%

import os
import shutil
import math

# Paths
base_dir = "crocDataaset"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")
val_dir = os.path.join(base_dir, "val")

# Subfolders
image_folder = "images"
label_folder = "labels"

# Create directories for test and val splits
for dir_path in [test_dir, val_dir]:
    os.makedirs(os.path.join(dir_path, image_folder), exist_ok=True)
    os.makedirs(os.path.join(dir_path, label_folder), exist_ok=True)

# Get list of all images and corresponding labels
image_files = sorted(os.listdir(os.path.join(train_dir, image_folder)))
label_files = sorted(os.listdir(os.path.join(train_dir, label_folder)))

# Ensure equal number of images and labels
assert len(image_files) == len(label_files), "Mismatch between images and labels count."

# Total dataset size
total_size = len(image_files)

# Compute split sizes using the floor
test_size = math.floor(total_size * 0.1)  # 10% for testing
val_size = math.floor(total_size * 0.2)   # 20% for validation
train_size = total_size - test_size - val_size  # Remaining for training

# Split datasets
test_images = image_files[:test_size]
test_labels = label_files[:test_size]

val_images = image_files[test_size:test_size + val_size]
val_labels = label_files[test_size:test_size + val_size]

train_images = image_files[test_size + val_size:]
train_labels = label_files[test_size + val_size:]

# Function to move files
def move_files(file_list, src_dir, dest_dir):
    for file_name in file_list:
        shutil.move(os.path.join(src_dir, file_name), os.path.join(dest_dir, file_name))

# Move test set
move_files(test_images, os.path.join(train_dir, image_folder), os.path.join(test_dir, image_folder))
move_files(test_labels, os.path.join(train_dir, label_folder), os.path.join(test_dir, label_folder))

# Move validation set
move_files(val_images, os.path.join(train_dir, image_folder), os.path.join(val_dir, image_folder))
move_files(val_labels, os.path.join(train_dir, label_folder), os.path.join(val_dir, label_folder))

print("Dataset split completed!")


Dataset split completed!


In [3]:
# Load model and train

from ultralytics import YOLO

model = YOLO("yolo11n.pt")  # load pretrained model
results = model.train(data=r"C:\Users\Raymond\Desktop\Machine Learning\crocDataaset\data.yaml", epochs=100) #train the fucking model


Ultralytics 8.3.39  Python-3.10.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:\Users\Raymond\Desktop\Machine Learning\crocDataaset\data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=Fal

train: Scanning C:\Users\Raymond\Desktop\Machine Learning\crocDataaset\train\labels... 138 images, 0 backgrounds, 0 corrupt: 100%|██████████| 138/138 [00:00<00:00, 845.51it/s]

train: New cache created: C:\Users\Raymond\Desktop\Machine Learning\crocDataaset\train\labels.cache



val: Scanning C:\Users\Raymond\Desktop\Machine Learning\crocDataaset\val\labels... 35 images, 0 backgrounds, 0 corrupt: 100%|██████████| 35/35 [00:00<00:00, 653.92it/s]

val: New cache created: C:\Users\Raymond\Desktop\Machine Learning\crocDataaset\val\labels.cache


...

    100/100      2.39G     0.4481     0.4639     0.9958         16        640: 100%|██████████| 9/9 [00:01<00:00,  5.73it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  8.15it/s]

                   all         35         36      0.952      0.772      0.808      0.538



100 epochs completed in 0.075 hours.
Optimizer stripped from runs\detect\train11\weights\last.pt, 5.5MB
Optimizer stripped from runs\detect\train11\weights\best.pt, 5.5MB

Validating runs\detect\train11\weights\best.pt...
Ultralytics 8.3.39  Python-3.10.11 torch-2.5.1+cu124 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  4.75it/s]


                   all         35         36      0.933      0.752      0.783      0.634
             Crocodile         31         32      0.889      0.754      0.817      0.561
                   Log          4          4      0.976       0.75      0.749      0.706
Speed: 0.3ms preprocess, 5.8ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\train11


In [9]:
# Detect using the model you created

from ultralytics import YOLO
import cv2

model = YOLO(r"runs\detect\train11\weights\best.pt") # default model location from your 
results = model(r"crocDataaset\test\images", save=True)


image 1/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\02HJTPB1BQQ0.jpg: 480x640 1 Crocodile, 1 Log, 13.1ms
image 2/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0FS2EPS5BRRI.jpg: 384x640 1 Crocodile, 31.0ms
image 3/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0H65CX45204L.jpg: 480x640 1 Crocodile, 14.0ms
image 4/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0OPN8TCQWAO4.jpg: 448x640 1 Crocodile, 15.0ms
image 5/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0PK6HUZ7JGFK.jpg: 448x640 1 Crocodile, 12.0ms
image 6/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0RBESQNMNOHU.jpg: 448x640 1 Crocodile, 23.0ms
image 7/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0UI650D8CASQ.jpg: 480x640 2 Crocodiles, 14.0ms
image 8/19 c:\Users\Raymond\Desktop\Machine Learning\crocDataaset\test\images\0Z3833WCBGMR.jpg: 448x640 2 Crocodiles, 15.0m